In [92]:
import requests
from owslib.ogcapi.features import Features
from pystac_client import Client

import obstore as obs
from obstore.store import from_url, AzureStore
from obstore.auth.planetary_computer import PlanetaryComputerCredentialProvider

from lonboard import viz
import geopandas as gpd
import jupytergis

import rasterio as rs
from rasterio.plot import show

In [93]:
url = "https://openveda.cloud/api/features/"

In [94]:
w = Features(url)
w.feature_collections()

['public.eis_fire_lf_perimeter_archive',
 'public.eis_fire_lf_newfirepix_archive',
 'public.eis_fire_lf_fireline_archive',
 'public.eis_fire_lf_fireline_nrt',
 'public.eis_fire_snapshot_fireline_nrt',
 'public.eis_fire_snapshot_newfirepix_nrt',
 'public.eis_fire_lf_newfirepix_nrt',
 'public.eis_fire_lf_perimeter_nrt',
 'public.eis_fire_snapshot_perimeter_nrt',
 'pg_temp.eis_fire_lf_perimeter_nrt_latest',
 'public.st_subdivide',
 'public.st_hexagongrid',
 'public.st_squaregrid']

In [95]:
perim = w.collection("public.eis_fire_snapshot_perimeter_nrt")

In [96]:
perim_q = w.collection_queryables("public.eis_fire_snapshot_perimeter_nrt")
perim_q["properties"]

{'geometry': {'$ref': 'https://geojson.org/schema/Geometry.json'},
 'duration': {'name': 'duration', 'type': 'number'},
 'farea': {'name': 'farea', 'type': 'number'},
 'fireid': {'name': 'fireid', 'type': 'number'},
 'flinelen': {'name': 'flinelen', 'type': 'number'},
 'fperim': {'name': 'fperim', 'type': 'number'},
 'geom_counts': {'name': 'geom_counts', 'type': 'string'},
 'isactive': {'name': 'isactive', 'type': 'number'},
 'low_confidence_grouping': {'name': 'low_confidence_grouping',
  'type': 'number'},
 'meanfrp': {'name': 'meanfrp', 'type': 'number'},
 'n_newpixels': {'name': 'n_newpixels', 'type': 'number'},
 'n_pixels': {'name': 'n_pixels', 'type': 'number'},
 'pixden': {'name': 'pixden', 'type': 'number'},
 'primarykey': {'name': 'primarykey', 'type': 'string'},
 'region': {'name': 'region', 'type': 'string'},
 't': {'name': 't', 'type': 'string'}}

In [97]:
new_mexico = ["-109", "31", "-103", "37"]

In [131]:
perim_results = w.collection_items(
    "public.eis_fire_snapshot_perimeter_nrt",
    bbox=new_mexico,
    limit=1000
)

In [132]:
perimeters = gpd.GeoDataFrame.from_features(perim_results["features"])
perimeters["region"].unique()

array(['CONUS'], dtype=object)

In [100]:
viz(perimeters)

/Users/will/DevSeed/NASA/disasters-hub/.venv/lib/python3.11/site-packages/lonboard/_geoarrow/ops/reproject.py:37: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

## Buildings

In [101]:
mpc_url = "https://planetarycomputer.microsoft.com/api/stac/v1/"
mpc = Client.open(mpc_url)
mpc.title

'Microsoft Planetary Computer STAC API'

In [102]:
mpc_search = mpc.collection_search(
    q="building"
)

print(f"{mpc_search.matched()} collections found")

/Users/will/DevSeed/NASA/disasters-hub/.venv/lib/python3.11/site-packages/pystac_client/collection_search.py:306: UserWarning: Server does not conform to COLLECTION_SEARCH. Filtering will be performed client-side where only bbox, datetime, and q arguments are supported
  warnings.warn(


2 collections found


In [103]:
for collection in mpc_search.collections():
    print(collection.id)

3dep-lidar-classification
ms-buildings


In [104]:
buildings_cat = mpc.get_collection("ms-buildings")
buildings_cat.title

'Microsoft Building Footprints'

In [105]:
building_regions_search = mpc.search(
    collections=["ms-buildings"],
    bbox=new_mexico
).item_collection_as_dict()

In [106]:
building_regions = gpd.GeoDataFrame.from_features(building_regions_search["features"])
building_regions

,geometry,title,datetime,description,table:columns,table:row_count,msbuildings:region,end_datetime,start_datetime
0,"POLYGON ((-66.95046 18.92146, -66.95046 71.360...",Building footprints,2022-07-06T00:00:00Z,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array', 'd...",129554779,United States,NaN,NaN
1,"POLYGON ((-66.95046 18.92146, -66.95046 71.360...",Building footprints,2022-06-14T00:00:00Z,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array'}]",129557523,United States,NaN,NaN
2,"POLYGON ((-86.71319 14.53409, -86.71319 32.716...",Building footprints,None,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array', 'd...",23769510,Mexico,2021-12-18T00:00:00+00:00,2017-06-10T00:00:00+00:00
3,"POLYGON ((-86.71319 14.53409, -86.71319 32.716...",Building footprints,None,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array'}]",23769510,Mexico,2021-12-18T00:00:00+00:00,2017-06-10T00:00:00+00:00
4,"POLYGON ((-61.00254 7.39233, -61.00254 32.4727...",Building footprints,None,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array', 'd...",3234,North America,2021-06-08T00:00:00+00:00,2014-06-19T00:00:00+00:00
5,"POLYGON ((-61.00254 7.39233, -61.00254 32.4727...",Building footprints,None,Parquet dataset with the building footprints,"[{'name': 'geometry', 'type': 'byte_array'}]",3234,North America,2021-06-08T00:00:00+00:00,2014-06-19T00:00:00+00:00


In [107]:
search = mpc.search(
    collections=["ms-buildings"],
    bbox=new_mexico,
    query={
        # "msbuildings:region": {"eq": "United States"},
        # "msbuildings:processing-date": {"eq": "2023-04-25"}
    },
)

ic = search.item_collection()

In [108]:
ic[0]

<Item id=United States_2022-07-06>

In [109]:
parquet_file_info = ic[0].assets["data"]

In [110]:
parquet_file_info

<Asset href=abfs://footprints/global/2022-07-06/ml-buildings.parquet/RegionName=United States>

In [111]:
authenticate_request = requests.get(url=f"https://planetarycomputer.microsoft.com/api/sas/v1/token/{buildings_cat.id}")
token = authenticate_request.json()["token"]

In [112]:
store = AzureStore(credential_provider=PlanetaryComputerCredentialProvider(parquet_file.href, account_name="bingmlbuildings"))

In [113]:
stream = obs.list(store)
for list_result in stream:
    print(list_result[0])

{'path': 'quadkey=002302201/part-00074-98ab98f9-b939-442b-a7e2-c8cccce09842.c000.snappy.parquet', 'last_modified': datetime.datetime(2022, 7, 14, 1, 32, 52, tzinfo=datetime.timezone.utc), 'size': 31925, 'e_tag': '0x8DA6538C538B8BB', 'version': None}
{'path': 'quadkey=021223333/part-00006-98ab98f9-b939-442b-a7e2-c8cccce09842.c000.snappy.parquet', 'last_modified': datetime.datetime(2022, 7, 14, 1, 33, 7, tzinfo=datetime.timezone.utc), 'size': 282863, 'e_tag': '0x8DA6538CE369652', 'version': None}
{'path': 'quadkey=021230301/part-00123-98ab98f9-b939-442b-a7e2-c8cccce09842.c000.snappy.parquet', 'last_modified': datetime.datetime(2022, 7, 14, 1, 33, 42, tzinfo=datetime.timezone.utc), 'size': 549519, 'e_tag': '0x8DA6538E295A204', 'version': None}
{'path': 'quadkey=021231203/part-00086-98ab98f9-b939-442b-a7e2-c8cccce09842.c000.snappy.parquet', 'last_modified': datetime.datetime(2022, 7, 14, 1, 34, 3, tzinfo=datetime.timezone.utc), 'size': 2422387, 'e_tag': '0x8DA6538EF2D7C87', 'version': None

In [114]:
path = "quadkey=023010213/part-00175-98ab98f9-b939-442b-a7e2-c8cccce09842.c000.snappy.parquet"

parquet_test = store.get(path).bytes()

with open("parquet_test.parquet", "wb") as f:
    f.write(parquet_test)

# reader_test = obs.open_reader(store, path)

In [115]:
test_gpd = gpd.read_parquet("parquet_test.parquet")
test_gpd

,geometry
0,"POLYGON ((-121.27386 38.03579, -121.27372 38.0..."
1,"POLYGON ((-121.6086 37.90019, -121.60842 37.90..."
2,"POLYGON ((-121.25965 37.95234, -121.25968 37.9..."
3,"POLYGON ((-121.21155 37.83008, -121.21154 37.8..."
4,"POLYGON ((-121.27115 37.813, -121.27103 37.813..."
...,...
266986,"POLYGON ((-121.32874 38.05675, -121.32875 38.0..."
266987,"POLYGON ((-121.35297 38.01904, -121.35297 38.0..."
266988,"POLYGON ((-121.08528 38.03919, -121.08528 38.0..."
266989,"POLYGON ((-121.54406 37.79019, -121.54409 37.7..."


## Population

In [116]:
pop_url = "https://data.worldpop.org/GIS/Population_Density/Global_2000_2020_1km_UNadj/2020/USA/usa_pd_2020_1km_UNadj.tif" # From https://data.humdata.org/dataset/worldpop-population-density-for-united-states-of-america

In [117]:
# population = rs.open(pop_url)

---

Trying to do it in a fully STAC way

In [118]:
stac_url = "https://openveda.cloud/api/stac"

In [119]:
veda_catalog = Client.open(stac_url)
veda_catalog.title

'VEDA (Visualization, Exploration, and Data Analysis) STAC API'

In [120]:
snapshot_perimeter = veda_catalog.collection_search(
    q="fire"
)

/Users/will/DevSeed/NASA/disasters-hub/.venv/lib/python3.11/site-packages/pystac_client/collection_search.py:306: UserWarning: Server does not conform to COLLECTION_SEARCH. Filtering will be performed client-side where only bbox, datetime, and q arguments are supported
  warnings.warn(


In [121]:
snapshot_perimeter.matched()

19

In [122]:
for collection in snapshot_perimeter.collections():
    print(collection.id)

barc-thomasfire
caldor-fire-behavior
disalexi-etsuppression
caldor-fire-burn-severity
eis_fire_perimeter
campfire-albedo-wsa-diff
frp-max-thomasfire
campfire-lst-day-diff
campfire-lst-night-diff
campfire-nlcd
hls-bais2-v2
landsat-nighttime-thermal
campfire-ndvi-diff
lis-tvegsuppression
lis-etsuppression
la-fires-hrrr-wind
la-fires-planet
la-fires-frp
la-fires-maxar
